# Face Detection

In [0]:
!pip install google-cloud-vision

In [0]:
from google.colab import files
files.upload()

In [0]:
!ls

In [0]:
path = r'face.jpg'

In [0]:
import os, io
from google.cloud import vision
from google.cloud.vision import types

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'service-account.json'

def starter(path):

    client = vision.ImageAnnotatorClient()
 
    if path.startswith('http') or path.startswith('gs:'):
        image = types.Image()
        image.source.image_uri = path
    else:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
        image = types.Image(content=content)
 
    result = client.face_detection(image=image).face_annotations
 
    return result


In [0]:
face = starter(path)
face

## Face expressions

In [0]:
# Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')

In [0]:
for f in face:
  print('Joy : {}'.format(likelihood_name[f.joy_likelihood]))
  print('Sorrow : {}'.format(likelihood_name[f.sorrow_likelihood]))
  print('Anger : {}'.format(likelihood_name[f.anger_likelihood]))
  print('Surprise : {}'.format(likelihood_name[f.surprise_likelihood]))


## Co-ordinates for face

In [0]:
from PIL import Image, ImageDraw


In [0]:
def helper_box(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)
    # Sepecify the font-family and the font-size
    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')
        # Place the confidence value/score of the detected faces above the
        # detection box in the output image
        draw.text(((face.bounding_poly.vertices)[0].x,
                   (face.bounding_poly.vertices)[0].y - 30),
                  str(format(face.detection_confidence, '.3f')) + '%',
                  fill='#FF0000')
    im.save(output_filename)